In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import os, os.path

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
start_time = time.time()

In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
sys.path.append('/home/hnoorazar/remote_sensing_codes/')

import remote_sensing_core as rc
import remote_sensing_core as rcp


In [3]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/" + \
           "01_NDVI_TS/04_Irrigated_eastern_Cloud70/Grant_2018_irrigated/"


In [4]:
####################################################################################
###
###      Parameters                   
###
####################################################################################
freedom_df = 7
Sav_win_size = 5
sav_order = 2

delt = 0.3
print ("delta = {fileShape}.".format(fileShape=delt))

delta = 0.3.


In [5]:
f_name = "Grant_2018_allFs_notCorrectYrs_70cloud.csv"
a_df = pd.read_csv(data_dir + f_name, low_memory=False)
a_df = rc.filter_out_nonIrrigated(a_df)
print ("After filtering out non-irrigated, a_df is of dimension {fileShape}.".format(fileShape=a_df.shape))


filter_NASS = False
filter_lastSurDate = False

if filter_NASS == True:
    if filter_lastSurDate == True:
        print ("1")
        last_part_name = "NassOut_CorrectYear"
    elif filter_lastSurDate == False:
        print ("2")
        last_part_name = "NassOut_NotCorrectYear"

if filter_NASS == False:
    if filter_lastSurDate == True:
        print ("3")
        last_part_name = "NassIn_CorrectYears"
    elif filter_lastSurDate == False:
        print ("4")
        last_part_name = "NassIn_NotCorrectYears"

print(last_part_name)
print ("filter_NASS is " + str(filter_NASS))
print ("filter_lastSurDate is " + str(filter_lastSurDate))


if (filter_NASS == True):
    a_df = rc.filter_by_lastSurvey(dt_df_surv = a_df, year=2018)
    print ("After filtering by last survey date, a_df is of dimension {fileShape}.".format(fileShape=a_df.shape))

if (filter_lastSurDate == True):
    a_df = rc.filter_out_NASS(dt_df_NASS = a_df)
    print ("After filtering out NASS, a_df is of dimension {fileShape}.".format(fileShape=a_df.shape))
######################
output_dir = data_dir + "/savitzky/Grant_Irrigated_EVI_2018_" + last_part_name + \
             "/delta" + str(delt) + "_Sav_win" + str(Sav_win_size) + "_Order"  + str(sav_order) + "/"

plot_dir_base = output_dir
print ("plot_dir_base is " + plot_dir_base)

os.makedirs(output_dir, exist_ok=True)
os.makedirs(plot_dir_base, exist_ok=True)

######################
a_df['year'] = 2018

# The following columns do not exist in the old data
#
if not('DataSrc' in a_df.columns):
    print ("Data source is being set to NA")
    a_df['DataSrc'] = "NA"

if not('CovrCrp' in a_df.columns):
    print ("Data source is being set to NA")
    a_df['CovrCrp'] = "NA"

a_df = rc.initial_clean_EVI(a_df)
a_df.head(2)
an_EE_TS = a_df.copy()

### List of unique polygons
polygon_list = an_EE_TS['geo'].unique()
print(len(polygon_list))

max_output_columns = ['ID', 'Acres', 'CovrCrp', 'CropGrp', 'CropTyp',
                      'DataSrc', 'ExctAcr', 'IntlSrD', 'Irrigtn', 'LstSrvD', 'Notes',
                      'RtCrpTy', 'Shap_Ar', 'Shp_Lng', 'TRS', 'county', 'year', 'geo',
                      'max_Doy', 'max_value', 'max_count']

all_poly_and_maxs_spline = pd.DataFrame(data=None, 
                                        index=np.arange(3*len(an_EE_TS)), 
                                        columns=max_output_columns)

all_poly_and_maxs_savitzky = pd.DataFrame(data=None, 
                                          index=np.arange(3*len(an_EE_TS)), 
                                          columns=max_output_columns)


min_output_columns = ['ID', 'Acres', 'CovrCrp', 'CropGrp', 'CropTyp',
                      'DataSrc', 'ExctAcr', 'IntlSrD', 'Irrigtn', 'LstSrvD', 'Notes',
                      'RtCrpTy', 'Shap_Ar', 'Shp_Lng', 'TRS', 'county', 'year', 'geo',
                      'min_Doy', 'min_value', 'min_count']

all_poly_and_mins_spline = pd.DataFrame(data=None, 
                                        index=np.arange(3*len(an_EE_TS)), 
                                        columns=min_output_columns)

all_poly_and_mins_savitzky = pd.DataFrame(data=None, 
                                          index=np.arange(3*len(an_EE_TS)), 
                                          columns=min_output_columns)

After filtering out non-irrigated, a_df is of dimension (1592272, 39).
4
NassIn_NotCorrectYears
filter_NASS is False
filter_lastSurDate is False
plot_dir_base is /Users/hn/Documents/01_research_data/remote_sensing/01_NDVI_TS/04_Irrigated_eastern_Cloud70/Grant_2018_irrigated//savitzky/Grant_Irrigated_EVI_2018_NassIn_NotCorrectYears/delta0.3_Sav_win5_Order2/
17706


In [6]:
a_poly = polygon_list[0]

In [7]:
curr_field = an_EE_TS[an_EE_TS['geo']==a_poly].copy()
################################################################
# Sort by DoY (sanitary check)
curr_field.sort_values(by=['doy'], inplace=True)

################################################################



In [8]:
year = int(curr_field['year'].unique())
plant = curr_field['CropTyp'].unique()[0]

# Take care of names, replace "/" and "," and " " by "_"
plant = plant.replace("/", "_")
plant = plant.replace(",", "_")
plant = plant.replace(" ", "_")
plant = plant.replace("__", "_")

county = curr_field['county'].unique()[0]
ID = curr_field['ID'].unique()[0]

### 
###  There is a chance that a polygon is repeated twice?
###

X = curr_field['doy'].copy()
y = curr_field['EVI'].copy()

In [9]:
my_savitzky_pred = rc.savitzky_golay(y, window_size=Sav_win_size, order=sav_order)

AttributeError: module 'remote_sensing_core' has no attribute 'savitzky_golay'

In [12]:
savitzky_pred = scipy.signal.savgol_filter(y, window_length = Sav_win_size, polyorder = sav_order)

In [13]:
len(my_savitzky_pred)

73

In [14]:
len(savitzky_pred)

73